# 微調 Open AI 模型

此筆記本基於 Open AI 目前在 [微調](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) 文件中提供的指引。

微調透過使用與特定用例或場景相關的額外資料和上下文重新訓練基礎模型，以提升其在您的應用中的表現。請注意，像是 _few shot learning_ 和 _retrieval augmented generation_ 這類提示工程技術，允許您使用相關資料來增強預設提示以提升品質。然而，這些方法受限於目標基礎模型的最大 token 視窗大小。

透過微調，我們實際上是使用所需資料重新訓練模型本身（允許我們使用比最大 token 視窗能容納更多的範例）— 並部署一個 _自訂_ 版本的模型，該版本在推論時不再需要提供範例。這不僅提升了我們提示設計的效能（我們在使用 token 視窗時有更多彈性），也可能降低成本（減少推論時需要傳送給模型的 token 數量）。

微調有四個步驟：
1. 準備訓練資料並上傳。
1. 執行訓練工作以獲得微調模型。
1. 評估微調模型並反覆調整以提升品質。
1. 在滿意後部署微調模型進行推論。

請注意，並非所有基礎模型都支援微調—[請參考 OpenAI 文件](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) 以取得最新資訊。您也可以微調先前已微調過的模型。在本教學中，我們將使用 `gpt-35-turbo` 作為微調的目標基礎模型。

---


### Step 1.1: 準備您的資料集

讓我們建立一個聊天機器人，透過用打油詩回答有關元素的問題，幫助您了解元素週期表。在_這個_簡單的教學中，我們將只建立一個資料集，用幾個範例回應來訓練模型，這些範例展示了資料的預期格式。在實際應用中，您需要建立一個包含更多範例的資料集。若有適用的公開資料集（針對您的應用領域），您也可以使用並重新格式化它以用於微調。

由於我們專注於`gpt-35-turbo`並尋求單輪回應（聊天完成），我們可以使用[此建議格式](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst)來建立範例，符合OpenAI聊天完成的要求。如果您預期多輪對話內容，則會使用[多輪範例格式](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst)，其中包含一個`weight`參數，用以標示哪些訊息應該在微調過程中使用（或不使用）。

在此教學中，我們將使用較簡單的單輪格式。資料採用[jsonl格式](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst)，每行一筆記錄，每筆記錄以JSON格式物件表示。以下片段展示了2筆記錄作為範例—完整範例集（10個範例）請參見[training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl)，這是我們用於微調教學的資料集。**注意：**每筆記錄_必須_定義在單行中（不可像格式化JSON檔案那樣分行）。

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

在實際應用中，您將需要更多範例以獲得良好結果—權衡點在於回應品質與微調所需的時間/成本。我們使用較小的資料集，以便快速完成微調並說明流程。請參考[這個OpenAI Cookbook範例](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst)以獲得更複雜的微調教學。


---

### 第 1.2 步 上傳您的資料集

使用 Files API [如此處所述](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file) 上傳資料。請注意，為了執行此程式碼，您必須先完成以下步驟：
 - 安裝 `openai` Python 套件（請確保您使用的版本 >=0.28.0 以獲得最新功能）
 - 將 `OPENAI_API_KEY` 環境變數設定為您的 OpenAI API 金鑰
欲了解更多資訊，請參閱課程提供的[設定指南](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst)。

現在，執行程式碼以從您的本地 JSONL 檔案建立一個上傳用的檔案。


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### Step 2.1: 使用 SDK 建立微調工作


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### 第 2.2 步：檢查工作的狀態

以下是您可以使用 `client.fine_tuning.jobs` API 執行的一些操作：
- `client.fine_tuning.jobs.list(limit=<n>)` - 列出最近的 n 個微調工作
- `client.fine_tuning.jobs.retrieve(<job_id>)` - 取得特定微調工作的詳細資訊
- `client.fine_tuning.jobs.cancel(<job_id>)` - 取消微調工作
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - 列出該工作最多 n 個事件
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

此流程的第一步是_驗證訓練檔案_，以確保資料格式正確。


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### 第 2.3 步：追蹤事件以監控進度


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### 步驟 2.4：在 OpenAI 儀表板查看狀態


您也可以透過造訪 OpenAI 網站並瀏覽平台的 _Fine-tuning_ 區段來查看狀態。這將顯示您目前工作的狀態，並讓您追蹤先前工作執行的歷史紀錄。在此截圖中，您可以看到先前的執行失敗，而第二次執行成功。作為背景說明，這是因為第一次執行時使用了格式錯誤的 JSON 檔案記錄 — 修正後，第二次執行成功完成，並使模型可供使用。

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.tw.png)


您也可以透過在視覺化儀表板中向下滾動來查看狀態訊息和指標，如下所示：

| Messages | Metrics |
|:---|:---|
| ![Messages](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.tw.png) |  ![Metrics](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.tw.png)|


---

### 第 3.1 步：在程式碼中取得 ID 並測試微調模型


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### 第 3.2 步：在 Playground 載入並測試微調模型

您現在可以透過兩種方式測試微調模型。首先，您可以造訪 Playground，並使用模型下拉選單從列出的選項中選擇您新微調的模型。另一個選項是使用微調面板中顯示的「Playground」選項（見上方截圖），它會啟動以下的 _比較_ 視圖，將基礎模型與微調模型版本並排顯示，方便快速評估。

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.tw.png)

只需填入您訓練資料中使用的系統上下文，並提供您的測試問題。您會注意到雙方都會更新相同的上下文和問題。執行比較後，您將看到它們輸出結果的差異。_請注意微調模型如何以您範例中提供的格式呈現回應，而基礎模型則僅遵循系統提示_。

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.tw.png)

您會注意到比較視圖還會提供每個模型的標記數量以及推論所花費的時間。**此特定範例為簡化示範，旨在展示流程，並不反映真實世界的資料集或情境**。您可能會發現兩個範例顯示相同的標記數（系統上下文和使用者提示相同），但微調模型推論所需時間較長（自訂模型）。

在真實世界情境中，您不會使用像這樣的玩具範例，而是針對真實資料（例如客戶服務的產品目錄）進行微調，回應品質將更為明顯。在 _那個_ 情境下，若要用基礎模型達到相同的回應品質，將需要更多自訂提示工程，這會增加標記使用量並可能延長推論處理時間。_想嘗試看看，請參考 OpenAI Cookbook 中的微調範例開始。_

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**免責聲明**：  
本文件係使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。雖然我們致力於確保翻譯的準確性，但請注意自動翻譯可能包含錯誤或不準確之處。原始文件的母語版本應視為權威來源。對於重要資訊，建議採用專業人工翻譯。我們不對因使用本翻譯而產生的任何誤解或誤譯負責。
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
